In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import sys
from matplotlib import pyplot as plt

In [ ]:
def load_images(path):
    folders = sorted(os.listdir(path))
    data = []
    for folder in folders:
        if(folder != "README"):
            images = sorted(os.listdir(os.path.join(path, folder)))
            for image in images:
                i = Image.open(os.path.join(path, os.path.join(folder, image)))
                i = i.convert("L")
                i = np.asarray(i, dtype = np.uint8)
                i = np.append(i, int(folder[1:]))
                data.append(i.tolist())
    return data

In [ ]:
from random import sample
def faceVSNonFace(path , label):
    folders = sorted(os.listdir(path))
    data = []
    for folder in folders:
        if(folder != "README" and folder != "person"):
            images = sorted(os.listdir(os.path.join(path, folder)))
            for image in images:
                i = Image.open(os.path.join(path, os.path.join(folder, image)))
                i = i.resize((92,112))
                i = i.convert("L")
                i = np.asarray(i, dtype = np.uint8)
                i = np.append(i, label)
                data.append(i.tolist())
    return data

In [ ]:
def plot(num,acc,x,y):
    plt.scatter(num, acc)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.show();

In [ ]:
def getFacesData():
    data = np.asarray(load_images('../input/att-database-of-faces'))
    data.shape
    training = data[1:][::2] # odd
    testing = data[0:][::2] # even
    print(training.shape)
    print(testing.shape)
    print(training[:10,:])
    xTraining = training[:,:-1]
    print(xTraining.shape)
    yTraining = training[:,-1]
    print(yTraining.shape)
    xTesting = testing[:,:-1]
    print(xTesting.shape)
    yTesting = testing[:,-1]
    print(yTesting.shape)
    return xTraining,yTraining,xTesting,yTesting

In [ ]:
from sklearn.model_selection import train_test_split
def splitData(nOfRows):
    facesData = np.asarray(faceVSNonFace('../input/att-database-of-faces' , 1))
    nonFacesData = sample(faceVSNonFace('../input/natural-images/natural_images' , 0) , nOfRows)
    #nonFacesData = np.asarray(faceVSNonFace('../input/natural-images/natural_images' , 0))
    nonFacesData = np.asarray(nonFacesData)
    # split both into train and test
    SEED = 0
    #faces
    xFace = facesData[:,:-1]
    yFace = facesData[:,-1]
    #non faces
    xNonFace = nonFacesData[:,:-1]
    yNonFace = nonFacesData[:,-1]

    xFaceTrain , xFaceTest, yFaceTrain , yFaceTest = train_test_split(xFace, yFace, test_size = 0.5, random_state = SEED)
    xNonFaceTrain , xNonFaceTest , yNonFaceTrain , yNonFaceTest = train_test_split(xNonFace, yNonFace, test_size = 0.5, random_state = SEED)

    xTrainData = np.concatenate((xFaceTrain, xNonFaceTrain), axis=0)
    yTrainData = np.concatenate((yFaceTrain, yNonFaceTrain), axis=0)

    xTestData = np.concatenate((xFaceTest, xNonFaceTest), axis=0)
    yTestData = np.concatenate((yFaceTest, yNonFaceTest), axis=0)

    # shuffle train data
    randomize = np.arange(len(xTrainData))
    np.random.shuffle(randomize)
    xTrainData = xTrainData[randomize]
    yTrainData = yTrainData[randomize]
    return xTrainData , yTrainData , xTestData , yTestData

In [ ]:
def Bonus():
    data = np.asarray(load_images('../input/att-database-of-faces'))
    data.shape
    training = []
    testing = []
    i=0
    while i<len(data):
        for j in range(7):
            training.append(data[i])
            i+=1
        for j in range(3):
            testing.append(data[i])
            i+=1
    training = np.asarray(training)
    testing = np.asarray(testing)
    print(training.shape)
    print(testing.shape)
    print(testing[:7,:])
    xTraining = training[:,:-1]
    print(xTraining.shape)
    yTraining = training[:,-1]
    print(yTraining.shape)
    xTesting = testing[:,:-1]
    print(xTesting.shape)
    yTesting = testing[:,-1]
    print(yTesting.shape)
    return xTraining,yTraining,xTesting,yTesting

In [ ]:
  def getEginValueAndVector(xTraining,xTesting):
    images_train=np.array(xTraining)
    images_test=np.array(xTesting)
    # get mean vecgtor of image train 
    mean_train = images_train.mean(axis=0)
    # get mean vecgtor of image test
    mean_test=images_test.mean(axis=0)
    Z_train = images_train - mean_train
    Z_test = images_test - mean_test
    cov = (1/len(images_train)) * np.dot(np.transpose(Z_train), Z_train) 
    value, vector = np.linalg.eigh(cov)
    idx=np.argsort(value)[::-1]
    value=value[idx]
    vector=vector[:,idx]
    return value,vector,Z_train,Z_test

def PCA(value,vector,Z_train,Z_test ,alpha):
        i=0
        valueDim=0
        sumValue=np.sum(value)
        ratio=0;
        for v in value:
            valueDim+=v
            ratio=valueDim/sumValue
            i+=1
            if(ratio>=alpha):
                break
        projection_matrix=vector[: , 0:i]
        new_dataSet_train=np.transpose(np.dot(np.transpose(projection_matrix),np.transpose(Z_train)))
        new_dataSet_test=np.transpose(np.dot(np.transpose(projection_matrix),np.transpose(Z_test)))
        return new_dataSet_train,new_dataSet_test

In [ ]:
#train pca over face dataset
alpha = [0.8,0.85,0.9,0.95]
nn=[1,3,5,7]
xTraining,yTraining,xTesting,yTesting=getFacesData()
value,vector,Z_train,Z_test=getEginValueAndVector(xTraining,xTesting)
print("N              Alpha         Accuracy")
for n in nn: 
    #Create KNN Classifier
    for a in alpha:
        x_train,x_test=PCA(value,vector,Z_train,Z_test,a)
        knn = KNeighborsClassifier(n_neighbors=n)
        #Train the model using the training sets
        knn.fit(x_train,yTraining)
        #Predict the response for test dataset
        y_pred = knn.predict(x_test)
        accuary=metrics.accuracy_score(yTesting, y_pred)
        print(f"{n}            {a}            {accuary}\n")

In [ ]:
#train pca over face VS nonfaces dataset
alpha = [0.8,0.85,0.9,0.95]
nn=[1,3,5,7]
numofRows=[300,400,500,600,700,800,900]
maxAccuarcy=[]
maxAcc=0
print("NumOfNonHuman              N               Alpha            Accuracy")
for num in numofRows:
    xTraining,yTraining,xTesting,yTesting=splitData(num)
    value,vector,Z_train,Z_test=getEginValueAndVector(xTraining,xTesting)
    maxAcc=-sys.maxsize - 1
    for n in nn: 
        #Create KNN Classifier
        for a in alpha:
            x_train,x_test=PCA(value,vector,Z_train,Z_test,a)
            knn = KNeighborsClassifier(n_neighbors=n)
            #Train the model using the training sets
            knn.fit(x_train,yTraining)
            #Predict the response for test dataset
            y_pred = knn.predict(x_test)
            accuary=metrics.accuracy_score(yTesting, y_pred)
            maxAcc=max(maxAcc,accuary)
            print(f"{num}            {n}               {a}              {accuary}\n")
    maxAccuarcy.append(maxAcc)
print(maxAccuarcy)    
plot(numofRows,maxAccuarcy,"Num of non faces","Accuarcy")

In [ ]:
#train pca over face dataset bonus
alpha = [0.8,0.85,0.9,0.95]
nn=[1,3,5,7]
xTraining,yTraining,xTesting,yTesting=Bonus()
value,vector,Z_train,Z_test=getEginValueAndVector(xTraining,xTesting)
print("N              Alpha         Accuracy")
for n in nn: 
    #Create KNN Classifier
    for a in alpha:
        x_train,x_test=PCA(value,vector,Z_train,Z_test,a)
        knn = KNeighborsClassifier(n_neighbors=n)
        #Train the model using the training sets
        knn.fit(x_train,yTraining)
        #Predict the response for test dataset
        y_pred = knn.predict(x_test)
        accuary=metrics.accuracy_score(yTesting, y_pred)
        print(f"{n}            {a}            {accuary}\n")

In [ ]:
import numpy as np
# number of dominant eigen vectors to be taken
dominantEigenVecotrsTaken = 12
def LDA(x,y):
    numberOfFeatures = x.shape[1]
    labels = np.unique(y)
    print(labels)
    # calculate Mu
    Mu = np.mean(x, axis=0)
    # initialize the between class and within class matrices "initially with zeroes so we can add on them"
    Sb = np.zeros((numberOfFeatures,numberOfFeatures))
    S = np.zeros((numberOfFeatures,numberOfFeatures))
    
    for c in labels:
        Di = x[y == c]
        Mui = np.mean(Di, axis=0)
        # Sb = sigma(ni*(Mui-M)*(Mui-M)Transpose)
        MeanDifference = (Mui - Mu).reshape(numberOfFeatures,1)
        Sb += Di.shape[0]*np.dot( MeanDifference , np.transpose(MeanDifference))
        #Sb += Di.shape[0]*np.dot( (Mui - Mu) , np.transpose(Mui-Mu))
        Zi = Di - np.transpose(Mui)
        Si = np.dot(np.transpose(Zi), Zi)
        S += Si   
    A = np.dot(np.linalg.inv(S),Sb)
    # get eigen vectors and values
    # what is the diffrence between eigh and eig ? 
    eigenValues , eigenVectors = np.linalg.eigh(A)
    #eigenValues , eigenVectors = np.linalg.eig(A)
    
    eigenVectors = np.transpose(eigenVectors)
    # sort in decreasing order
    idxs = np.argsort(abs(eigenValues))[::-1]
    eigenValues = eigenValues[idxs]
    eigenVectors = eigenVectors[idxs]
    # return the first n eigenVectors in our case we need the 39th dominant eigenVectors
    return eigenVectors[0 : dominantEigenVecotrsTaken]

def Project(DataMatrix , ProjectionMatrix):
    return np.dot(DataMatrix , np.transpose(ProjectionMatrix))


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

def analyze_model(trained_model, prediction,x_test, y_test , k):
    print(f'k={k}: accuracy={metrics.accuracy_score(y_test, prediction )} ')
    
def knn_classifier (x_train, y_train, x_test, y_test):
    kArray = [1,3,5,7]
    for i in kArray:
        #classifier = classifier = GridSearchCV(KNeighborsClassifier(), {
        #    'n_neighbors': (1,9,2)
        #})
        classifier = KNeighborsClassifier(n_neighbors = i)
        # fit the data to train the model
        trained = classifier.fit(x_train, y_train)
        # find y_predicted.
        predicted = classifier.predict(x_test)
        #print(classifier.best_params_)
        # report analysis and draw the confusion matrix
        analyze_model(trained, predicted, x_test, y_test , i)



In [ ]:
#LDA training
xTraining,yTraining,xTesting,yTesting=getFacesData()
ProjectionMatrix = LDA(xTraining , yTraining)
ProjectionMatrix.shape
    
xProjectedTrain = Project(xTraining, ProjectionMatrix)
xProjectedTest = Project(xTesting, ProjectionMatrix)
xProjectedTrain.shape
knn_classifier(xProjectedTrain , yTraining , xProjectedTest , yTesting)

In [ ]:
# analyze success and failure cases

xTraining,yTraining,xTesting,yTesting=splitData(400)
ProjectionMatrix = LDA(xTraining , yTraining)
ProjectionMatrix.shape
xProjectedTrain = Project(xTraining, ProjectionMatrix)
xProjectedTest = Project(xTesting, ProjectionMatrix)
xProjectedTrain.shape

In [ ]:
classifier = KNeighborsClassifier(n_neighbors = 1)

trained = classifier.fit(xProjectedTrain, yTraining)

predicted = classifier.predict(xProjectedTest)

xCorrect = []
xFalse = []
for i in range(len(predicted)):
    if predicted[i]==yTesting[i]:
        xCorrect.append(xTesting[i])
    else:
        xFalse.append(xTesting[i])
print(np.asarray(xCorrect).shape , np.asarray(xFalse).shape)
xCorrect = np.asarray(xCorrect)
xFalse = np.asarray(xFalse)


In [ ]:
from matplotlib import pyplot as plt
import random
temp = random.choice(xCorrect)
temp = np.reshape(temp , (112,92))
plt.imshow(temp , cmap='gray')
plt.show()

In [ ]:
#Faces VS Non Faces
numofRows=[300,400,500,600,700]
for n in numofRows:
    xTraining,yTraining,xTesting,yTesting=splitData(n)
    ProjectionMatrix = LDA(xTraining , yTraining)
    ProjectionMatrix.shape
    xProjectedTrain = Project(xTraining, ProjectionMatrix)
    xProjectedTest = Project(xTesting, ProjectionMatrix)
    xProjectedTrain.shape
    print(f'for n = {n}')
    knn_classifier(xProjectedTrain , yTraining , xProjectedTest , yTesting)

In [ ]:
#Bonus 
xTraining,yTraining,xTesting,yTesting = Bonus()
ProjectionMatrix = LDA(xTraining , yTraining)
ProjectionMatrix.shape
    
xProjectedTrain = Project(xTraining, ProjectionMatrix)
xProjectedTest = Project(xTesting, ProjectionMatrix)
xProjectedTrain.shape
knn_classifier(xProjectedTrain , yTraining , xProjectedTest , yTesting)